# <font color='gray'> Bibliotecas necessárias
---

In [ ]:
import requests
from io import BytesIO
import polars as pl
import pandas as pd

# <font color='gray'> Carregando os dados
----

Este projeto utiliza dados do Sistema de Informações sobre Mortalidade (SIM) do [SIM - DataSUS](https://opendatasus.saude.gov.br/pt_BR/dataset/sim), disponível no OpenDataSUS. O SIM é a principal base de dados de óbitos no Brasil e reúne informações detalhadas sobre mortes registradas no país, incluindo causa básica do óbito, características da vítima e circunstâncias da morte.

Iniciamos as análises com dados de 2023.

In [ ]:
# URL do arquivo CSV
url = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SIM/DO23OPEN.csv"

# Faz o download do arquivo
response = requests.get(url)
response.raise_for_status()  # Garante que a requisição foi bem-sucedida

# Carrega o CSV diretamente para o Polars
df = pl.read_csv(
    BytesIO(response.content),
    separator=";",  # Define o delimitador correto
    quote_char='"',  # Considera aspas como parte do valor
    encoding="utf8",
    ignore_errors=True  # Ignora linhas problemáticas (opcional)
)
print(f"Linhas: {df.height}, Colunas: {df.width}")


In [ ]:
# dados 
df

In [ ]:
# Calculando a porcentagem de nulos por coluna
perc_nulos = (df.null_count() / df.height) * 100

# Transpondo as colunas para linhas (usando melt para reestruturar)
perc_nulos_transposto = perc_nulos.melt()

# Exibindo as colunas que contém dados nulos a partir de 60%
lista_col = perc_nulos_transposto.filter(pl.col('value') > 60)["variable"].to_list()
lista_col

# <font color='gray'> Análise Exploratória dos Dados
---

In [ ]:
deletar = """
 14  SERIESCFAL  404622 non-null   object
 20  IDADEMAE    29183 non-null    object
 21  ESCMAE      28950 non-null    object
 22  ESCMAE2010  28820 non-null    object
 23  SERIESCMAE  12375 non-null    object
 24  OCUPMAE     25908 non-null    object
 25  QTDFILVIVO  29164 non-null    object
 26  QTDFILMORT  28333 non-null    object
 27  GRAVIDEZ    29909 non-null    object
 28  SEMAGESTAC  28698 non-null    object
 29  GESTACAO    28698 non-null    object
 30  PARTO       29785 non-null    object
 31  OBITOPARTO  29545 non-null    object
 32  PESO        28705 non-null    object
 33  TPMORTEOCO  87151 non-null    object
 34  OBITOGRAV   87679 non-null    object
 35  OBITOPUERP  87662 non-null    object
 37  EXAME       4106 non-null     object
 38  CIRURGIA    3949 non-null     object
 46  CB_PRE      0 non-null        object
 47  COMUNSVOIM  257792 non-null   object
 49  CIRCOBITO   153897 non-null   object
 50  ACIDTRAB    51838 non-null    object
 51  FONTE       126366 non-null   object
 53  DTINVESTIG  242831 non-null   object
 60  FONTEINV    245325 non-null   object
 65  CAUSAMAT    25 non-null       object
 66  ESCMAEAGR1  28820 non-null    object
 71  NUDIASOBCO  60100 non-null    object
 72  DTCADINV    60520 non-null    object
 73  TPOBITOCOR  60520 non-null    object
 74  DTCONINV    60032 non-null    object
 75  FONTES      29861 non-null    object
 76  TPRESGINFO  2629 non-null     object
 77  TPNIVELINV  60588 non-null    object
 78  DTCADINF    29861 non-null    object
 79  MORTEPARTO  29861 non-null    object
 80  DTCONCASO   27375 non-null    object
 81  ALTCAUSA    28851 non-null    object
 39  CAUSAMAT    25 non-null       object
 40  ESCMAEAGR1  28820 non-null    object
 45  NUDIASOBCO  60100 non-null    object
 46  DTCADINV    60520 non-null    object
 47  TPOBITOCOR  60520 non-null    object
 48  DTCONINV    60032 non-null    object
 49  FONTES      29861 non-null    object
 50  TPRESGINFO  2629 non-null     object
 51  TPNIVELINV  60588 non-null    object
 52  DTCADINF    29861 non-null    object
 53  MORTEPARTO  29861 non-null    object
 54  DTCONCASO   27375 non-null    object
 55  ALTCAUSA    28851 non-null    object
 57  TPPOS       947635 non-null   object
 58  TP_ALTERA   5161 non-null     object
 59  CB_ALT      4741 non-null     object
"""
deletar = {li.strip().split()[1].strip() for li in deletar.split('\n') if li.strip() != ''}

In [ ]:
for col in deletar:
    # print(col)
    try:
        df.drop(col, inplace=True, axis=1)
    except KeyError:
        pass

In [ ]:
deletar2 = [
    'NUMEROLOTE',
    'ORIGEM',
    'contador',
    'TIPOBITO',
    'NATURAL',
    'CODMUNNATU',
    'IDADE',
    'CODESTAB',
    'DTATESTADO',
    'DTCADASTRO',
    'ATESTANTE',
    'STCODIFICA',
    'CODIFICADO',
    'VERSAOSIST',
    'VERSAOSCB',
    'DTRECEBIM',
    'DTRECORIGA',
    'OPOR_DO',
    'ESCFALAGR1',
    'STDOEPIDEM',
    'STDONOVA',
    'DIFDATA',
]

In [ ]:
for col in deletar2:
    # print(col)
    try:
        df.drop(col, inplace=True, axis=1)
    except KeyError:
        pass

In [ ]:
df.info()

In [ ]:
codigos = [
    f'X{i}' for i in range(85, 100)
] + [
    f'Y0{i}' for i in range(0, 10)
]
codigos = set(codigos)
codigos

In [ ]:
# incluir coluna para estado
# incluir coluna para cidade
# transformar datas para formato iso

In [ ]:
cols_cid = [
    'LINHAA',
    'LINHAB',
    'LINHAC',
    'LINHAD',
    'LINHAII',
    'CAUSABAS',
    'ATESTADO',
    'CAUSABAS_O',
]
tem = lambda colunas: any(((col is not None) and (cod in col)) for col in colunas for cod in codigos)
df2 = df[df[cols_cid].apply(tem, axis=1)]

In [ ]:
df2.SEXO.value_counts()